# Pre-processing of GLAMOS MB data:

## Setting up:

In [ ]:
import pandas as pd
import os
import warnings
from tqdm.notebook import tqdm
import re
import massbalancemachine as mbm
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
from scipy.spatial.distance import cdist
import pyproj
import matplotlib.pyplot as plt
import seaborn as sns


from scripts.helpers import *
from scripts.glamos_preprocess import *

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

## 1. Transform .dat files to .csv:

In [ ]:
# Get all files with pmb (for winter and annual mb):
glamosfiles_mb_a, glamosfiles_mb_w = [], []
for file in os.listdir(path_PMB_GLAMOS_a_raw):
    # check if current path is a file
    if os.path.isfile(os.path.join(path_PMB_GLAMOS_a_raw, file)):
        glamosfiles_mb_a.append(file)

for file in os.listdir(path_PMB_GLAMOS_w_raw):
    # check if current path is a file
    if os.path.isfile(os.path.join(path_PMB_GLAMOS_w_raw, file)):
        glamosfiles_mb_w.append(file)

print('Examples of index stake raw files:\n', glamosfiles_mb_a[:5])

# Transform all files to csv
RUN = True
if RUN:
    emptyfolder(path_PMB_GLAMOS_csv_a)
    emptyfolder(path_PMB_GLAMOS_csv_w)
    for file in glamosfiles_mb_a:
        fileName = re.split('.dat', file)[0]
        processDatFile(fileName, path_PMB_GLAMOS_a_raw, path_PMB_GLAMOS_csv_a)

    for file in glamosfiles_mb_w:
        fileName = re.split('.dat', file)[0]
        processDatFile(fileName, path_PMB_GLAMOS_w_raw, path_PMB_GLAMOS_csv_w)

    # Separate clariden into clariden II and III
    fileName = 'clariden_annual.csv'
    clariden_csv_a = pd.read_csv(path_PMB_GLAMOS_csv_a + fileName,
                                 sep=',',
                                 header=0,
                                 encoding='latin-1')
    clariden_csv_a[clariden_csv_a['# name'] == 'L'].to_csv(
        path_PMB_GLAMOS_csv_a + 'claridenL_annual.csv', index=False)
    clariden_csv_a[clariden_csv_a['# name'] == 'U'].to_csv(
        path_PMB_GLAMOS_csv_a + 'claridenU_annual.csv', index=False)

    fileName = 'clariden_winter.csv'
    clariden_csv_w = pd.read_csv(path_PMB_GLAMOS_csv_w + fileName,
                                 sep=',',
                                 header=0,
                                 encoding='latin-1')
    clariden_csv_w[clariden_csv_w['# name'] == 'L'].to_csv(
        path_PMB_GLAMOS_csv_w + 'claridenL_winter.csv', index=False)
    clariden_csv_w[clariden_csv_w['# name'] == 'U'].to_csv(
        path_PMB_GLAMOS_csv_w + 'claridenU_winter.csv', index=False)

    os.remove(path_PMB_GLAMOS_csv_a + 'clariden_annual.csv')
    os.remove(path_PMB_GLAMOS_csv_w + 'clariden_winter.csv')

# Example:
fileName = 'aletsch_annual.csv'
aletsch_csv = pd.read_csv(path_PMB_GLAMOS_csv_a + fileName,
                          sep=',',
                          header=0,
                          encoding='latin-1')
aletsch_csv.head(2)

##  2. Measurement periods:
### 2.1. Annual measurements: 
Process annual measurements and put all stakes into one csv file

In [ ]:
# Assemble all into one csv file:
RUN = True
if RUN:
    # Annual:
    df_annual_raw = pd.DataFrame()
    for file in tqdm(os.listdir(path_PMB_GLAMOS_csv_a), desc='Annual stakes'):
        fileName = re.split('.csv', file)[0]
        glacierName = re.split('_', fileName)[0]
        df = pd.read_csv(path_PMB_GLAMOS_csv_a + file,
                         sep=',',
                         header=0,
                         encoding='latin-1')
        df['glacier'] = glacierName
        df['period'] = 'annual'

        # Correct years and add hydrol. year:
        df_processed = transformDates(df)

        # Remove obvious duplicates:
        df_processed = df_processed.drop_duplicates()

        # Transform to lat/lon system
        df_processed = LV03toWGS84(df_processed)

        df_annual_raw = pd.concat([df_annual_raw, df_processed])

    # Get the year (end of hydr. year):
    df_annual_raw['YEAR'] = df_annual_raw['date1'].apply(
        lambda x: pd.to_datetime(x).year)

# Reshape to WGMS format:
# re-order columns:
df_annual_raw = df_annual_raw[[
    'YEAR', '# name', 'glacier', 'date0', 'date1', 'lat', 'lon', 'height',
    'mb_we', 'period', 'date_fix0', 'date_fix1', 'time0', 'time1',
    'date_quality', 'position_quality', 'mb_raw', 'density', 'density_quality',
    'measurement_quality', 'measurement_type', 'mb_error', 'reading_error',
    'density_error', 'error_evaluation_method', 'source'
]]
df_annual_raw.rename(columns={
    '# name': 'POINT_ID',
    'lat': 'POINT_LAT',
    'lon': 'POINT_LON',
    'height': 'POINT_ELEVATION',
    'date0': 'FROM_DATE',
    'date1': 'TO_DATE',
    'mb_we': 'POINT_BALANCE',
    'glacier': 'GLACIER',
    'period': 'PERIOD'
},
                     inplace=True)
# remove duplicates:
df_annual_raw = df_annual_raw.drop_duplicates()
df_annual_raw.head(2)

In [ ]:
# verify that to_date is always from_date year + 1
# drop rows where this is not the case:
rows_to_drop = []
for i, row in df_annual_raw.iterrows():
    if pd.to_datetime(row['TO_DATE'], format='%Y%m%d').year - pd.to_datetime(
            row['FROM_DATE'], format='%Y%m%d').year != 1:
        # print(row['FROM_DATE'], row['TO_DATE'])
        # drop row:
        rows_to_drop.append(i)
df_annual_raw.drop(rows_to_drop, inplace=True)

### 2.2. Winter measurements:
For each point in annual meas., take winter meas that was taken closest:

In [ ]:
winter_glaciers = [
    re.split('_winter.csv', f)[0] for f in os.listdir(path_PMB_GLAMOS_csv_w)
]
RUN = False
if RUN:
    emptyfolder(path_PMB_GLAMOS_csv_w_clean)
    for glacier in tqdm(df_annual_raw.GLACIER.unique(),
                        desc='Glaciers',
                        leave=False,
                        position=0):
        # If glacier does not have winter measurements:
        if glacier not in winter_glaciers:
            continue

        # Get winter all PMB measurements (a lot more than annual)
        df_winter = pd.read_csv(path_PMB_GLAMOS_csv_w + glacier +
                                '_winter.csv',
                                sep=',',
                                header=0,
                                encoding='latin-1')
        df_winter['period'] = 'winter'
        df_winter['glacier'] = glacier

        # Correct years and add hydrol. year:
        df_winter = transformDates(df_winter)

        # Remove obvious duplicates:
        df_winter = df_winter.drop_duplicates()

        # Transform to lat/lon system
        df_winter = LV03toWGS84(df_winter)

        # Add the year:
        df_winter['YEAR'] = df_winter['date1'].apply(
            lambda x: pd.to_datetime(x).year)

        # Reshape to WGMS format:
        df_winter.rename(columns={
            '# name': 'POINT_ID',
            'lat': 'POINT_LAT',
            'lon': 'POINT_LON',
            'height': 'POINT_ELEVATION',
            'date0': 'FROM_DATE',
            'date1': 'TO_DATE',
            'mb_we': 'POINT_BALANCE',
            'glacier': 'GLACIER',
            'period': 'PERIOD'
        },
                         inplace=True)
        # re-order columns:
        df_winter = df_winter[[
            'YEAR', 'POINT_ID', 'GLACIER', 'FROM_DATE', 'TO_DATE', 'POINT_LAT',
            'POINT_LON', 'POINT_ELEVATION', 'POINT_BALANCE', 'PERIOD',
            'date_fix0', 'date_fix1', 'time0', 'time1', 'date_quality',
            'position_quality', 'mb_raw', 'density', 'density_quality',
            'measurement_quality', 'measurement_type', 'mb_error',
            'reading_error', 'density_error', 'error_evaluation_method',
            'source'
        ]]

        winter_indices = []
        # Iterate over all rows in annual stakes:
        for index in tqdm(range(
                len(df_annual_raw[df_annual_raw.GLACIER == glacier])),
                          desc='rows',
                          leave=False,
                          position=1):
            pointA = df_annual_raw[df_annual_raw.GLACIER ==
                                   glacier].iloc[index]
            yearA = pointA.YEAR

            # Filter winter to same year as point:
            df_winter_year = df_winter[df_winter.YEAR == yearA]

            # If no winter measurement was taken that year:
            if len(df_winter_year) == 0:
                continue

            # Calculate closest winter point to annual stake:
            df_winter_year['point'] = [(x, y) for x, y in zip(
                df_winter_year['POINT_LAT'], df_winter_year['POINT_LON'])]
            pointA['point'] = (pointA.POINT_LAT, pointA.POINT_LON)
            closest = closest_point(pointA['point'],
                                    list(df_winter_year['point']))

            # Convert to LAEA Europe
            x_laea, y_laea = latlon_to_laea(closest[0], closest[1])
            x_laea_point, y_laea_point = latlon_to_laea(
                pointA.POINT_LAT, pointA.POINT_LON)

            # Calculate distance:
            distance_m = np.sqrt((x_laea_point - x_laea)**2 +
                                 (y_laea_point - y_laea)**2)

            # Only keep winter stakes that are within 10m:
            if distance_m > 10:
                continue
            else:
                winter_indices.append(df_winter_year[df_winter_year['point'] ==
                                                     closest].index[0])

        # Save winter stake:
        df_winter_stakes = df_winter.iloc[winter_indices]
        df_winter_stakes.to_csv(path_PMB_GLAMOS_csv_w_clean + glacier +
                                '_winter.csv')

### 2.3. Assemble both periods:

In [ ]:
df_all_raw = df_annual_raw.copy()
for file in os.listdir(path_PMB_GLAMOS_csv_w_clean):
    fileName = re.split('.csv', file)[0]
    glacierName = re.split('_', fileName)[0]
    df_winter = pd.read_csv(path_PMB_GLAMOS_csv_w_clean + file,
                            sep=',',
                            header=0,
                            encoding='latin-1').drop(columns='Unnamed: 0')

    df_all_raw = pd.concat([df_all_raw, df_winter])

print('Number of winter and annual samples:', len(df_all_raw))
print('Number of winter samples:',
      len(df_all_raw[df_all_raw.PERIOD == 'winter']))
print('Number of annual samples:',
      len(df_all_raw[df_all_raw.PERIOD == 'annual']))

# download all stakes coordinates:
# df_all_raw[['GLACIER', 'POINT_ID', 'POINT_LAT', 'POINT_LON',
#             'PERIOD']].to_csv(path_PMB_GLAMOS_csv + 'coordinates_all.csv')

df_all_raw.reset_index(drop=True, inplace=True)

In [ ]:
# Check for issues with dates:
for i, row in df_all_raw.iterrows():
    if pd.to_datetime(row['TO_DATE'], format='%Y%m%d').year - pd.to_datetime(
            row['FROM_DATE'], format='%Y%m%d').year != 1:
        print(row['GLACIER'], row['PERIOD'], row['FROM_DATE'],
              pd.to_datetime(row['FROM_DATE'], format='%Y%m%d').year,
              row['TO_DATE'],
              pd.to_datetime(row['TO_DATE'], format='%Y%m%d').year)

In [ ]:
# See above that the problem is that for some winter measurements the FROM_DATE is the same year as the TO_DATE (even same date)
# Correct it by setting it to beginning of hydrological year:
for index, row in df_all_raw.iterrows():
    if row['PERIOD'] == 'winter':
        df_all_raw.loc[index, 'FROM_DATE'] = str(
            pd.to_datetime(row['TO_DATE'], format='%Y%m%d').year - 1) + '1001'

# Check again:
for i, row in df_all_raw.iterrows():
    if pd.to_datetime(row['TO_DATE'], format='%Y%m%d').year - pd.to_datetime(
            row['FROM_DATE'], format='%Y%m%d').year != 1:
        print(row['GLACIER'], row['PERIOD'], row['FROM_DATE'],
              pd.to_datetime(row['FROM_DATE'], format='%Y%m%d').year,
              row['TO_DATE'],
              pd.to_datetime(row['TO_DATE'], format='%Y%m%d').year)

# Save all stakes:
df_all_raw.to_csv(path_PMB_GLAMOS_csv + 'df_all_raw.csv')

## 3. Add RGIs Ids:

In [ ]:
# RGI Ids:
# Read rgi ids:
path_rgi = '../../../data/GLAMOS/CH_glacier_ids_long.csv'
rgi_df = pd.read_csv(path_rgi, sep=',')
rgi_df.rename(columns=lambda x: x.strip(), inplace=True)
rgi_df.sort_values(by='short_name', inplace=True)
rgi_df.set_index('short_name', inplace=True)
rgi_df.head(2)

In [ ]:
# Keep important features:
df_all_raw = pd.read_csv(path_PMB_GLAMOS_csv + 'df_all_raw.csv',
                         sep=',',
                         header=0,
                         encoding='latin-1').drop(columns='Unnamed: 0')
df_pmb = df_all_raw[[
    'YEAR',
    'POINT_ID',
    'GLACIER',
    'FROM_DATE',
    'TO_DATE',
    'POINT_LAT',
    'POINT_LON',
    'POINT_ELEVATION',
    'POINT_BALANCE',
    'PERIOD',
]]
df_pmb.head(3)

In [ ]:
# Add RGIs:
# Specify the shape filename of the glaciers outline obtained from RGIv6
glacier_outline_fname = '../../../data/GLAMOS/nsidc0770_11.rgi60.CentralEurope/11_rgi60_CentralEurope.shp'

# Load the target data and the glacier outlines
glacier_outline = gpd.read_file(glacier_outline_fname)

# Add RGI IDs through intersection with shapefiles:
df_pmb = mbm.data_processing.utils.get_rgi(data=df_pmb,
                                           glacier_outlines=glacier_outline)

# Add RGIs without intersections (by finding the closest polygon):
# for points where polygon intersection is NaN (about a 1000)
no_match_df = df_pmb[df_pmb.RGIId.isna()]
geometry = [
    Point(lon, lat)
    for lon, lat in zip(no_match_df["POINT_LON"], no_match_df["POINT_LAT"])
]
points_gdf = gpd.GeoDataFrame(no_match_df,
                              geometry=geometry,
                              crs=glacier_outline.crs)
for index in tqdm(no_match_df.index):
    point = points_gdf.loc[index]['geometry']
    polygon_index = glacier_outline.distance(point).sort_values().index[0]
    closest_rgi = glacier_outline.loc[polygon_index].RGIId
    df_pmb.at[index, 'RGIId'] = closest_rgi

In [ ]:
# Look at identified RGIs per glacier:
rgiids6 = df_pmb[['GLACIER',
                  'RGIId']].sort_values(by='GLACIER').drop_duplicates()
rgis = {}
for gl in rgiids6.GLACIER.unique():
    rgis[gl] = list(rgiids6[rgiids6.GLACIER == gl].RGIId)
rgis

In [ ]:
# Manual pre-processing and removal of errors:
# Silvretta: weird outlier coordinate
df_pmb_clean = df_pmb.copy()
index_outlier = df_pmb_clean[(df_pmb_clean.GLACIER == 'silvretta')
                             & (df_pmb_clean.POINT_LAT > 46.9)].index
df_pmb_clean.drop(index_outlier, inplace=True)

# and remove the stake that is on the neighbouring glacier:
index_outlier = df_pmb_clean[(df_pmb_clean.GLACIER == 'silvretta')
                             & (df_pmb_clean.RGIId != 'RGI60-11.00804')].index
df_pmb_clean.drop(index_outlier, inplace=True)

# Albigna: different rgis, remove stakes that are for two neighbouring glaciers:
index_outlier = df_pmb_clean[(df_pmb_clean.GLACIER == 'albigna')
                             & (df_pmb_clean.RGIId != 'RGI60-11.02285')].index
df_pmb_clean.drop(index_outlier, inplace=True)

# Err glacier: remove stakes that are on neighbouring glacier:
index_outlier = df_pmb_clean[(df_pmb_clean.GLACIER == 'err')
                             & (df_pmb_clean.RGIId != 'RGI60-11.01516')].index
df_pmb_clean.drop(index_outlier, inplace=True)

# Gries: weird outlier coordinate
index_outlier = df_pmb_clean[(df_pmb_clean.GLACIER == 'gries')
                             & (df_pmb_clean.RGIId != 'RGI60-11.01876')].index
df_pmb_clean.drop(index_outlier, inplace=True)

# Limmern: three stakes on neighbouring glacier
index_outlier = df_pmb_clean[(df_pmb_clean.GLACIER == 'limmern')
                             & (df_pmb_clean.RGIId != 'RGI60-11.00918')].index
df_pmb_clean.drop(index_outlier, inplace=True)

# Offental: on no RGI v6 outline
df_pmb_clean = df_pmb_clean[df_pmb_clean.GLACIER != 'ofental']

# Orny: change to correct RGIId
index_outlier = df_pmb_clean[(df_pmb_clean.GLACIER == 'orny')].index
for i in index_outlier:
    df_pmb_clean.at[i, 'RGIId'] = 'RGI60-11.02775'

# Plattalva: change to correct RGIId
index_outlier = df_pmb_clean[(df_pmb_clean.GLACIER == 'plattalva')].index
for i in index_outlier:
    df_pmb_clean.at[i, 'RGIId'] = 'RGI60-11.00892'

# Rhone outliers outside of RGI:
index_outlier = df_pmb_clean[(df_pmb_clean.GLACIER == 'rhone')
                             & (df_pmb_clean.POINT_LAT < 46.58)].index
df_pmb_clean.drop(index_outlier, inplace=True)

# Blauschnee, not in any RGI v6 outline:
df_pmb_clean = df_pmb_clean[df_pmb_clean.GLACIER != 'blauschnee']

# Remove unteraar
df_pmb_clean = df_pmb_clean[df_pmb_clean.GLACIER != 'unteraar']

# Look at identified RGIs per glacier:
rgiids6 = df_pmb_clean[['GLACIER',
                        'RGIId']].sort_values(by='GLACIER').drop_duplicates()
rgis = {}
for gl in rgiids6.GLACIER.unique():
    rgis[gl] = list(rgiids6[rgiids6.GLACIER == gl].RGIId)
print('RGIs after pre-processing:')
rgis

In [ ]:
# save to csv:
df_pmb_clean.to_csv(path_PMB_GLAMOS_csv + 'df_pmb_all.csv')

In [ ]:
df_pmb_clean.GLACIER.unique()

## 5. Cut from 1961:

In [ ]:
# Filter to start of MS data (1961) or ERA5-Land data (1950):
df_pmb_60s = df_pmb_clean[df_pmb_clean.YEAR > 1960].sort_values(
    by=['GLACIER', 'YEAR'], ascending=[True, True])
print('Number of winter and annual samples:', len(df_pmb_60s))
print('Number of annual samples:',
      len(df_pmb_60s[df_pmb_60s.PERIOD == 'annual']))
print('Number of winter samples:',
      len(df_pmb_60s[df_pmb_60s.PERIOD == 'winter']))

In [ ]:
# Number of measurements per year:
df_pmb_60s.groupby(['YEAR', 'PERIOD']).size().unstack().plot(kind='bar',
                                                             stacked=True,
                                                             figsize=(15, 5))
plt.title('Number of total measurements per year since 1961')

In [ ]:
num_gl = df_pmb_60s.groupby(['GLACIER']).size().sort_values()
num_gl.plot(kind='bar', figsize=(15, 5))
plt.title('Number of total measurements per glacier since 1961')

In [ ]:
# Plot number of measurements per year
# Number of measurements per glacier per year:
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
num_gl_yr = df_pmb_60s[df_pmb_60s.GLACIER == 'claridenL'].groupby(
    ['YEAR', 'PERIOD']).size().unstack().reset_index()
num_gl_yr.plot(x='YEAR',
               kind='bar',
               stacked=True,
               ax=ax,
               title=f'{glacierName}')
ax.set_ylabel('Number of measurements')
ax.set_title(f'Number of measurements per year: {glacierName}', fontsize=14)
plt.tight_layout()

### 5.1. Merge stakes that are close: 


In [ ]:
# clean df_pmb_60s
# Group similar stakes:
df_pmb_60s_clean = pd.DataFrame()
for gl in tqdm(df_pmb_60s.GLACIER.unique(), desc='glacier', position=0):
    print('----------------\n', gl, ':\n----------------')
    df_gl = df_pmb_60s[df_pmb_60s.GLACIER == gl]
    df_gl_cleaned = remove_close_points(df_gl)
    df_pmb_60s_clean = pd.concat([df_pmb_60s_clean, df_gl_cleaned])

# save to csv:
df_pmb_60s_clean.drop(['x', 'y', 'point'], axis=1).to_csv(
    path_PMB_GLAMOS_csv + 'df_pmb_60s_clean.csv', index=False)

# save coordinates:
df_pmb_60s_clean[['GLACIER', 'POINT_ID', 'POINT_LAT', 'POINT_LON', 'PERIOD'
                  ]].to_csv(path_PMB_GLAMOS_csv + 'coordinate_60s_clean.csv')

In [ ]:
# Number of measurements per glacier per year:
num_gl_yr = df_pmb_60s_clean.groupby(['GLACIER', 'YEAR', 'PERIOD'
                                      ]).size().unstack().reset_index()

# Plot one glacier per column:
big_gl = num_gl[num_gl > 250].index.sort_values()
num_glaciers = len(big_gl)
fig, ax = plt.subplots(num_glaciers, 1, figsize=(15, 5 * num_glaciers))
for i, gl in enumerate(big_gl):
    num_gl_yr[num_gl_yr.GLACIER == gl].plot(x='YEAR',
                                            kind='bar',
                                            stacked=True,
                                            ax=ax[i],
                                            title=gl)
    ax[i].set_ylabel('Number of measurements')
    ax[i].set_title(gl, fontsize=14)
plt.tight_layout()

## Add topographical information from OGGM:

In [ ]:
# Provide the column name for the column that has the RGI IDs for each of the stakes
# dataset = mbm.Dataset(data=df_pmb_clean,
#                       region_name='CH',
#                       data_path='')
# # Specify the topographical features of interest
# # Please see the OGGM documentation what variables are available: https://oggm.org/tutorials/stable/notebooks/10minutes/machine_learning.html ('topo', 'slope_factor', 'dis_from_border')
# voi_topographical = ['aspect', 'slope']

# # Retrieve the topographical features for each stake measurement and add them to the dataset
# dataset.get_topo_features(vois=voi_topographical)

# Switch to oggm datapulling notebook:
df_pmb_topo = pd.read_csv(path_PMB_GLAMOS_csv + 'CH_wgms_dataset.csv')

# Change from mm w.e. to m w.e.:
df_pmb_topo['POINT_BALANCE'] = df_pmb_topo['POINT_BALANCE'] / 1000

print('Number of winter and annual samples:', len(df_pmb_topo))
print('Number of annual samples:',
      len(df_pmb_topo[df_pmb_topo.PERIOD == 'annual']))
print('Number of winter samples:',
      len(df_pmb_topo[df_pmb_topo.PERIOD == 'winter']))
df_pmb_topo.head(2)

In [ ]:
# Histogram of mass balance
df_pmb_topo['POINT_BALANCE'].hist(bins=20)

## Give new stake IDs:
Give new stake IDs with glacier name and then a number according to the elevation. This is because accross glaciers some stakes have the same ID which is not practical.

In [ ]:
for glacierName in tqdm(df_pmb_topo.GLACIER.unique(), desc = 'glaciers'):
    gl_data = df_pmb_topo[df_pmb_topo.GLACIER == glacierName]
    stakeIDS = gl_data.groupby('POINT_ID')[[
        'POINT_LAT', 'POINT_LON', 'POINT_ELEVATION'
    ]].mean()
    stakeIDS.reset_index(inplace=True)
    # Change the ID according to elevation
    new_ids = stakeIDS[['POINT_ID',
                        'POINT_ELEVATION']].sort_values(by='POINT_ELEVATION')
    new_ids['POINT_ID_new'] = [
        f'{glacierName}_{i}' for i in range(len(new_ids))
    ]
    for i, row in new_ids.iterrows():
        df_pmb_topo.loc[(df_pmb_topo.GLACIER == glacierName) &
                        (df_pmb_topo.POINT_ID == row.POINT_ID),
                        'POINT_ID'] = row.POINT_ID_new
# Save to csv:
df_pmb_topo.to_csv(path_PMB_GLAMOS_csv + 'CH_wgms_dataset.csv', index=False)


In [ ]:
# Histogram of mass balance
df_pmb_topo['POINT_BALANCE'].hist(bins=20)

## Glacier wide MB:

In [ ]:
# Get all files with pmb (for winter and annual mb):
glamosfiles_smb = []
for file in os.listdir(path_SMB_GLAMOS_raw):
    # check if current path is a file
    if os.path.isfile(os.path.join(path_SMB_GLAMOS_raw,
                                   file)) and 'obs' in file:
        glamosfiles_smb.append(file)
print('Examples of index stake raw files:\n', glamosfiles_smb[:5])
# Transform all files to csv
RUN = True
if RUN:
    emptyfolder(path_SMB_GLAMOS_csv + 'obs/')
    for file in glamosfiles_smb:
        fileName = re.split('.dat', file)[0]
        processDatFileGLWMB(fileName, path_SMB_GLAMOS_raw,
                            path_SMB_GLAMOS_csv + 'obs/')

# Example:
fileName = 'aletsch_obs.csv'
aletsch_csv = pd.read_csv(path_SMB_GLAMOS_csv + 'obs/' + fileName,
                          sep=',',
                          header=0,
                          encoding='latin-1')
aletsch_csv.head(2)

In [ ]:
# Get all files with pmb (for winter and annual mb):
glamosfiles_smb = []
for file in os.listdir(path_SMB_GLAMOS_raw):
    # check if current path is a file
    if os.path.isfile(os.path.join(path_SMB_GLAMOS_raw,
                                   file)) and 'fix' in file:
        glamosfiles_smb.append(file)
print('Examples of index stake raw files:\n', glamosfiles_smb[:5])
# Transform all files to csv
RUN = True
if RUN:
    emptyfolder(path_SMB_GLAMOS_csv + 'fix/')
    for file in glamosfiles_smb:
        fileName = re.split('.dat', file)[0]
        processDatFileGLWMB(fileName, path_SMB_GLAMOS_raw,
                            path_SMB_GLAMOS_csv + 'fix/')

# Example:
fileName = 'aletsch_fix.csv'
aletsch_csv = pd.read_csv(path_SMB_GLAMOS_csv + 'fix/' + fileName,
                          sep=',',
                          header=0,
                          encoding='latin-1')
aletsch_csv.head(2)